# Debugging autoreload

In [ ]:
%load_ext autoreload
%autoreload 2

# Load packages

In [ ]:
from pytorch_tabular.utils import load_covertype_dataset
from rich.pretty import pprint
from sklearn.model_selection import BaseCrossValidator, ParameterGrid, ParameterSampler
import torch
import pickle
import shutil
from scipy import stats
import shap
from sklearn.model_selection import RepeatedStratifiedKFold
from glob import glob
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import itertools
from sklearn.model_selection import train_test_split
import numpy as np
from pytorch_tabular.utils import make_mixed_dataset, print_metrics
from pytorch_tabular import available_models
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig, GANDALFConfig, TabNetModelConfig, FTTransformerConfig, DANetConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig
from pytorch_tabular.models.common.heads import LinearHeadConfig
from pytorch_tabular.tabular_model_tuner import TabularModelTuner
from sklearn.metrics import mean_absolute_error
from pytorch_tabular import MODEL_SWEEP_PRESETS
import pandas as pd
from pytorch_tabular import model_sweep
from src.pt.model_sweep import model_sweep_custom
import warnings
from src.utils.configs import read_parse_config
from src.pt.hyper_opt import train_hyper_opt
from src.utils.hash import dict_hash
import pathlib
from tqdm import tqdm
import distinctipy
import matplotlib.patheffects as pe
import matplotlib.colors as mcolors
from statannotations.Annotator import Annotator
from scipy.stats import mannwhitneyu
from plottable import ColumnDefinition, Table
from plottable.plots import bar
from plottable.cmap import normed_cmap, centered_cmap
import optuna
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.cm
import matplotlib as mpl
from statsmodels.stats.multitest import multipletests
import re
from itertools import chain
from pathlib import Path
import requests
from matplotlib.ticker import MaxNLocator
from regression_bias_corrector import LinearBiasCorrector
from scipy.stats import mannwhitneyu, variation, levene
from copy import deepcopy


def make_rgb_transparent(rgb, bg_rgb, alpha):
    return [alpha * c1 + (1 - alpha) * c2 for (c1, c2) in zip(rgb, bg_rgb)]

def form_bar(base):
    def formatter(x):
        return f'{str(int(round(x * base)))}/{base}'
    return formatter


# Process raw data

In [ ]:
path = f"E:/YandexDisk/Work/bbd/mriya"

df_info = pd.read_excel(f"{path}/Испытуемые Яндекс.xlsx", sheet_name='Info', index_col=0)
df_info.rename(columns={'sex': 'Sex'}, inplace=True)
df_info.insert(3, 'Age', (df_info['sample_date'] - df_info['birthday']) / np.timedelta64(1, 'D') / 365.25)
df_info = df_info[df_info['Age'].notna()]

df_ifa_transform = pd.read_excel(f"{path}/data/ИФА/transformation.xlsx", index_col=0)
ifa_feats = df_ifa_transform.index.to_list()

df_ifa_raw = pd.read_excel(f"{path}/data/ИФА/raw.xlsx", skiprows=list(range(1, 14)))

for f in ifa_feats[1::]:
    if not df_ifa_raw[f"{ifa_feats[0]} Samples"].equals(df_ifa_raw[f"{f} Samples"]):
        print(f"{ifa_feats[0]} differs from {f}")

for f in ifa_feats:
    df_ifa_raw[f"log({f} Undiluted)"] = df_ifa_transform.at[f, 'Slope'] * np.log(df_ifa_raw[f"{f} Raw"]) + df_ifa_transform.at[f, 'Intercept']
    df_ifa_raw[f"{f}"] = np.exp(df_ifa_raw[f"log({f} Undiluted)"]) * df_ifa_transform.at[f, 'Dilution']
    df_ifa_raw[f"log({f})"] = np.log(df_ifa_raw[f"{f}"])
    
df_ifa_raw.set_index(f"{ifa_feats[0]} Samples", inplace=True)
df_ifa_raw.loc[:, list(itertools.chain.from_iterable([[f"{f} Raw", f"log({f} Undiluted)", f"{f}", f"log({f})"] for f in ifa_feats]))].to_excel(f"{path}/data/ИФА/data.xlsx", index_label='Samples')

n_rows = 2
n_cols = 4
fig_height = 10
fig_width = 18

sns.set_theme(style='ticks')
fig, axs = plt.subplots(n_rows, n_cols, figsize=(fig_width, fig_height), gridspec_kw={}, sharey=False, sharex=False, layout='constrained')

for f_id, f in enumerate(ifa_feats):
    df_ifa_control_points = pd.read_excel(f"{path}/data/ИФА/feats_control_points.xlsx", sheet_name=f, index_col=0)
    row_id, col_id = divmod(f_id, n_cols)
    df_ifa_raw[f"log(x)"] = np.log(df_ifa_raw[f"{f} Raw"])
    df_ifa_raw[f"log(y)"] = df_ifa_raw[f"log({f} Undiluted)"]
    scatter = sns.scatterplot(
        data=df_ifa_raw,
        x=f"log(x)",
        y=f"log(y)",
        linewidth=0.25,
        alpha=0.75,
        edgecolor="k",
        s=20,
        color='gray',
        ax=axs[row_id, col_id],
        zorder=1,
    )
    scatter = sns.scatterplot(
        x=df_ifa_control_points["log(x)"].values,
        y=df_ifa_control_points["log(y)"].values,
        linewidth=0.25,
        alpha=0.75,
        edgecolor="k",
        s=40,
        color='red',
        ax=axs[row_id, col_id],
        zorder=2,
    )
    axs[row_id, col_id].set_title(f, fontsize='large')
    # axs[n_rows - 1, n_cols - 1].axis('off') 
fig.savefig(f"{path}/data/ИФА/fit.png", bbox_inches='tight', dpi=200)
fig.savefig(f"{path}/data/ИФА/fit.pdf", bbox_inches='tight')
plt.close(fig)

# Check correlation with age

In [ ]:
path = f"E:/YandexDisk/Work/bbd/mriya"

df_info = pd.read_excel(f"{path}/Испытуемые Яндекс.xlsx", sheet_name='Info', index_col=0)
df_info.rename(columns={'sex': 'Sex'}, inplace=True)
df_info.insert(3, 'Age', (df_info['sample_date'] - df_info['birthday']) / np.timedelta64(1, 'D') / 365.25)
df_info = df_info[df_info['Age'].notna()]

df_ifa = pd.read_excel(f"{path}/data/ИФА/data.xlsx", index_col=0)
df_ifa.loc[df_ifa.index, 'Age'] = df_info.loc[df_ifa.index, 'Age']

df_ifa_transform = pd.read_excel(f"{path}/data/ИФА/transformation.xlsx", index_col=0)
ifa_feats = df_ifa_transform.index.to_list()

groups_feats = {
    'feats': ifa_feats,
    'log(feats)': [f"log({f})" for f in ifa_feats]
}

for gf in groups_feats:

    df_corr = pd.DataFrame(index=groups_feats[gf], columns=['rho'])
    for f in tqdm(groups_feats[gf]):
        df_tmp = df_ifa.loc[:, ['Age', f]].dropna(axis=0, how='any')
        if df_tmp.shape[0] > 1:
            vals_1 = df_tmp.loc[:, 'Age'].values
            vals_2 = df_tmp.loc[:, f].values
            df_corr.at[f, 'rho'], _ = stats.pearsonr(vals_1, vals_2)
    df_corr.dropna(axis=0, how='any', inplace=True)
    df_corr.insert(1, "abs(rho)", df_corr['rho'].abs())
    df_corr.sort_values(["abs(rho)"], ascending=[False], inplace=True)
    feats_cnt_wo_age = df_corr.index.to_list()
    feats_cnt = ['Age'] + feats_cnt_wo_age
    df_corr = df_corr.apply(pd.to_numeric)

    sns.set_theme(style='ticks')
    fig, ax = plt.subplots(figsize=(0.5 + 0.3 * df_corr.index.str.len().max(), 0.5 + 0.4 * len(feats_cnt_wo_age) + 0.04 * df_corr.index.str.len().max()) , layout='constrained')
    heatmap = sns.heatmap(
        df_corr.loc[:, ['rho']],
        annot=True,
        fmt=".2f",
        vmin=-1.0,
        vmax=1.0,
        cmap='coolwarm',
        linewidth=0.1,
        linecolor='black',
        #annot_kws={"fontsize": 15},
        cbar_kws={
            # "shrink": 0.9,
            # "aspect": 30,
            #'fraction': 0.046, 
            #'pad': 0.04,
        },
        ax=ax
    )
    heatmap_pos = ax.get_position()
    ax.figure.axes[-1].set_position([heatmap_pos.x1 + 0.05, heatmap_pos.y0, 0.1, heatmap_pos.height])
    ax.figure.axes[-1].set_ylabel(r"Pearson $\rho$")
    for spine in ax.figure.axes[-1].spines.values():
        spine.set(visible=True, lw=0.25, edgecolor="black")
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set(xticklabels=[])
    ax.set(xticks=[])
    plt.savefig(f"{path}/data/ИФА/age_{gf}_pearsonr.png", bbox_inches='tight', dpi=200)
    plt.savefig(f"{path}/data/ИФА/age_{gf}_pearsonr.pdf", bbox_inches='tight')
    plt.close(fig)